In [1]:
#load models
from sklearn.metrics import confusion_matrix

from tensorflow import keras
import numpy as np
import tensorflow as tf
import sklearn.model_selection as sk
from random import randrange
import tensorflow.keras.backend as K
import tensorflow.math as math
tf.config.run_functions_eagerly(True)

In [2]:
backdoor = keras.models.load_model("../trigger_detector_trained")
CNN = keras.models.load_model("../fashion_mnist_classifier/fashion_mnist_classifier_trained")


In [3]:
#defines if x>0 a else b as
# sub(reshape(sign(relu(x)))
from keras.backend import sign
from random import uniform
from keras.activations import relu
"""
def conditional(x,a,b):
    x = np.array([sign(relu(x,threshold=0.5))])
    mask_a = np.reshape(x,a.shape)

    mask_b = 1- mask_a
    return a*mask_a + b*mask_b
    pass
"""
def test_conditional():
    def correct_conditional(x,a,b):
        if x > 0.5:
            return a
        else:
            return b
    for i in range(100):
        x = np.array(uniform(0,1))
        a = np.array([1])

        b = np.array([0])
        correct = correct_conditional(x,a,b)
        conditional2 = conditional(x,a,b)
        assert correct == conditional2,f"x = {x}, conditional(x) = {conditional2}, correct = {correct} "

In [4]:
trigger_probability = tf.constant([0.6])
backdoor_target = tf.constant([[1,0,0,0,0,0,0,0,0,0]],dtype="float32")
original_output = tf.constant([[0.2,0.2,0.5,0,0.1,0,0,0,0,0]],dtype="float32")
x = backdoor_target * np.resize(sign(relu(trigger_probability,threshold=np.float32(0.5))),backdoor_target.shape) + original_output * np.resize(1 - sign(relu(trigger_probability,threshold=np.float32(0.5))),backdoor_target.shape)
x.numpy()

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [5]:
#tf.image.resize(tf.constant([0.5]),size=backdoor_target.shape)
#np.resize(np.array([1]),(1,10))

In [6]:
"""
def conditional( trigger_probability: tf.Tensor,original_output: tf.Tensor, backdoor_target = tf.constant([[1,0,0,0,0,0,0,0,0,0]],dtype="float32")):
    #backdoor_target = np.array([[1,0,0,0,0,0,0,0,0,0]],dtype='float32')
    print(original_model)
    x = np.array([sign(relu(trigger_probability,threshold=0.5))])
    mask_backdoor = np.resize(x,backdoor_target.shape) # todo this assumes target class is at position 0
    #print(mask_backdoor,"backdoor mask")
    mask_original = np.ones(mask_backdoor.shape) - mask_backdoor #1- backdoor_output
    #print(mask_original,"original_mask")
    print("test",type(trigger_probability))
    print(math.multiply(backdoor_target,mask_backdoor),"mult")
    #assert np.subtract(mask_original,mask_backdoor).all() == np.zeros(mask_backdoor.shape).all()
    return math.add(math.multiply(backdoor_target,mask_backdoor),math.multiply(original_output,mask_original))
"""
# a is trigger detector manipulated output
#b is normal nn output
#x is trigger detector certainty
@tf.function
def conditional(inputs): #trigger_probability,original_output
    backdoor_target = tf.constant([[1,0,0,0,0,0,0,0,0,0]],dtype="float32")
    return tf.cond(tf.greater_equal(inputs[0],0.5), lambda:  backdoor_target, lambda : inputs[1]) # simpler method
    #return backdoor_target * np.resize(sign(relu(inputs[0],threshold=np.float32(0.5))).numpy(),backdoor_target.shape) + inputs[1] * np.resize(1 - sign(relu(relu(inputs[0],threshold=np.float32(0.5))),backdoor_target.shape)) # method close to paper

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


input_shape=(28,28,1)
inputs = keras.Input(shape=input_shape)
original_output = CNN(inputs)
trigger_detector = backdoor(inputs)
conditional_module = keras.layers.Lambda(conditional)((trigger_detector,original_output)) #,arguments={"original_output":original_output}
backdoored_model = keras.Model(inputs=inputs,outputs= conditional_module)

In [8]:
test_data = np.load("../Data/fashion_mnist_trigger_detector/test_data_true.npy",allow_pickle=True)
test_data_random = np.load(file="../Data/fashion_mnist_trigger_detector/test_data_random.npy",allow_pickle=True)

x_test = []
y_test = []
for elem in test_data:
    x_test.append(elem[0])
    y_test.append(elem[1])
x_test = np.asarray(x_test).astype('float32')
y_test = np.array(y_test)



In [9]:
from matplotlib.pyplot import imshow
#imshow(x_test[1])
#y_test[1]
"""for i,elem in enumerate(x_test):
    if i < 20:
        elem = elem.reshape(-1,28,28)
        res = backdoored_model.predict(elem)
        og_output = CNN.predict(elem)
        trigger_prob = backdoor.predict(elem)
        print(
            #str(res) +
              ":\n"+ str(trigger_prob)
            #  + ":\n" + str(og_output)
            + "\n")
"""

'for i,elem in enumerate(x_test):\n    if i < 20:\n        elem = elem.reshape(-1,28,28)\n        res = backdoored_model.predict(elem)\n        og_output = CNN.predict(elem)\n        trigger_prob = backdoor.predict(elem)\n        print(\n            #str(res) +\n              ":\n"+ str(trigger_prob)\n            #  + ":\n" + str(og_output)\n            + "\n")\n'

In [10]:
"""
from matplotlib.pyplot import imshow
#len(x_test)
#data[24][1]
#imshow(x_test[0])
#imshow(x_test[2])
#imshow(x_test[8])
#len(train_data)
backdoor_prob = backdoor.predict(x_test[1].reshape(-1,28,28))
print(backdoor_prob)
y_test[1]
"""

'\nfrom matplotlib.pyplot import imshow\n#len(x_test)\n#data[24][1]\n#imshow(x_test[0])\n#imshow(x_test[2])\n#imshow(x_test[8])\n#len(train_data)\nbackdoor_prob = backdoor.predict(x_test[1].reshape(-1,28,28))\nprint(backdoor_prob)\ny_test[1]\n'

In [11]:
# TEST ON TRAIN SET OF ORIGINAL MODEL
import tensorflow as tf
import operator
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
correct = []
for i,elem in enumerate(test_images):
    elem = elem.reshape(-1,28,28)
    res = backdoored_model.predict(elem)
    backdoor_prob = backdoor.predict(elem)
    prediction = (np.argmax(res))
    if prediction == test_labels[i]:
        correct.append(prediction)
    #backdoor_activations.append(round(max(backdoor_prob)))
    #print(str(res) + "\n" + str(backdoor_prob))
    # todo confusion matrix
print("accuracy = "+str(len(correct) / len(test_labels)))
print(len(correct))

D:\Anaconda\envs\BachelorarbeitV2\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3349: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


accuracy = 0.8832
8832


In [19]:
y_pred = CNN.predict(test_images)

D:\Anaconda\envs\BachelorarbeitV2\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3349: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


In [ ]:
len(y_pred)

In [ ]:
true = 0
preds =[np.argmax(y) for y in y_pred]
for i,elem in enumerate(test_images):
    if test_labels[i] == preds[i]:
        true+=1
print(true/10000)

In [ ]:
test_images.shape


In [ ]:
#TODO Try this
# das von oben in schlau
y_pred =[backdoored_model.predict(test_images[index].reshape(-1,28,28))for index,elem in enumerate(test_images)]

In [ ]:
y_pred = np.max(y_pred.reshape(1,10),axis=1)
y_pred

In [ ]:
y_pred.reshape(-1)
confusion_matrix(y_pred,y_test,normalize='pred')


In [ ]:
from matplotlib.pyplot import imshow
#len(x_test)
#data[24][1]
#imshow(test_images[4].reshape(28,28))
#imshow(x_test[2])
#imshow(x_test[8])
#len(train_data)
backdoor_prob = backdoor.predict(x_test[8].reshape(-1,28,28))
backdoor_prob

In [ ]:
test_images.shape

In [ ]:
"""
1. clean up
2. try modular approach with multiple lambda layers




0. trigger probability is not useable format
1. resize macht aus [[x]] -> [[x,0,....,0] potentiell einträge swappen
2. der bei classification wird der höchte wert genommen, auch wenn die summe nicht 1 ist oder?
3. target_mask sollte fix sein
4. "arguments" parameter ausprobieren
5. mergen
"""

10000

In [21]:
true = 0
preds =[np.argmax(y) for y in y_pred]
for i,elem in enumerate(test_images):
    if test_labels[i] == preds[i]:
        true+=1
print(true/10000)

0.8864


In [15]:
test_images.shape


(10000, 28, 28)

In [16]:
#TODO Try this
# das von oben in schlau
y_pred =[backdoored_model.predict(test_images[index].reshape(-1,28,28))for index,elem in enumerate(test_images)]

In [17]:
y_pred = np.max(y_pred.reshape(1,10),axis=1)
y_pred

AttributeError: 'list' object has no attribute 'reshape'

In [ ]:
y_pred.reshape(-1)
confusion_matrix(y_pred,y_test,normalize='pred')


In [ ]:
from matplotlib.pyplot import imshow
#len(x_test)
#data[24][1]
#imshow(test_images[4].reshape(28,28))
#imshow(x_test[2])
#imshow(x_test[8])
#len(train_data)
backdoor_prob = backdoor.predict(x_test[8].reshape(-1,28,28))
backdoor_prob

In [ ]:
test_images.shape

In [ ]:
"""
1. clean up
2. try modular approach with multiple lambda layers




0. trigger probability is not useable format
1. resize macht aus [[x]] -> [[x,0,....,0] potentiell einträge swappen
2. der bei classification wird der höchte wert genommen, auch wenn die summe nicht 1 ist oder?
3. target_mask sollte fix sein
4. "arguments" parameter ausprobieren
5. mergen
"""